# **Stop Words & Tokenizing(2021)**

In [1]:
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# DB 연결
# !pip install pymysql
import pymysql

# DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

# 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

# 텍스트 분석
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re

# 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

# 모델 저장 및 로드
import joblib

# 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
maeil = pd.read_csv('../../../../Code/Data/2021_maeil.csv')
asia = pd.read_csv('../../../../Code/Data/2021_asia.csv')

In [3]:
maeil[maeil['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,score,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed


In [4]:
asia[asia['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,score,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed
6323,sk하이닉스,660,아시아경제,2021012907,"[속보]SK하이닉스, 작년 매출 31.9조원…영업이익 5조원",https://view.asiae.co.kr/article/2021012907373...,NaN,2021-01-29,7,0,128500,128500,121500,122500,7391307,-0.004065,-1,0
6324,sk하이닉스,660,아시아경제,2021012907,"[속보]SK하이닉스, 작년 4분기 매출 7.9조원…영업이익 9659억원",https://view.asiae.co.kr/article/2021012907434...,NaN,2021-01-29,7,0,128500,128500,121500,122500,7391307,-0.004065,-1,0


In [5]:
maeil.dropna(axis=0, inplace=True)   
maeil = maeil.reset_index(drop=True)   # reset_index() 꼭 해줘야 함...!!
asia.dropna(axis=0, inplace=True)
asia = asia.reset_index(drop=True)     # reset_index() 꼭 해줘야 함...!!

In [6]:
media_DF_list = [maeil, asia]

In [7]:
# Stop Words List에 각 매체명 추가
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt")
media_list = ['매일경제', '아시아경제', '삼프로TV', '슈카월드', '한국경제TV']
for word in [media_list]:
    stopwords.append(word)

In [8]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [9]:
for i in range(len(media_DF_list)):
    media_DF_list[i]['except_stopwords'] = 0
    for j in range(len(media_DF_list[i]['text'])):
#         hangeul = re.compile('[^ ㄱ-ㅣ 가-힣]')                  # 정규 표현식  → 한글 추출 규칙 : 띄어쓰기(1개)를 포함한 한글
#         result = hangeul.sub('', media_DF_list[i]['text'][j])   # 위에 설정한 hangeul 규칙을 text에 적용
        okt = Okt()                                             # 형태소 추출
        nouns = okt.nouns(media_DF_list[i]['text'][j])
        nouns = [x for x in nouns if len(x) > 1]                # 한 글자 키워드 제거
        nouns = [x for x in nouns if x not in stopwords]        # 불용어 제거
        
        corpus = " ".join(nouns)                                # List를 String으로 변환
        media_DF_list[i]['except_stopwords'][j] = corpus

In [10]:
maeil.to_csv('../../../../Code/Data/Test/2021_maeil.csv', index=False)
asia.to_csv('../../../../Code/Data/Test/2021_asia.csv', index=False)